In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import regex as re

In [2]:
df = pd.read_csv('./preprocessed_data/data.csv',index_col=[0])

In [3]:
df['cn'] = df['cn'].apply(lambda x: x.replace('\t',''))
df['cn'] = df['cn'].apply(lambda x: x.replace('\xa0',''))

In [4]:
# 조 살려서 자르기
df['pre_cn'] = df['cn']
# 조 
pattern = '\\n([제|0-9 ]{1,}조[\(|가-힣\) |A-Z|a-z|0-9|,m]{1,})'
for i in range(len(df)):
    text = df['pre_cn'][i]
    matchedList = re.findall(pattern,text)
    for match in matchedList:
        text = text.replace(match,'\n?'+match+'\n')
    df['pre_cn'][i] = text

C:\Users\hyj\AppData\Local\Temp\ipykernel_4352\1750625931.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pre_cn'][i] = text


In [5]:
df['article'] = df['pre_cn'].apply(lambda x:x.split('?'))

In [6]:
df['pre_cn'] = df['pre_cn'].apply(lambda x: x.replace('\n',''))
df['sent_jo'] = df['pre_cn'].apply(lambda x:x.split('?'))
df['sent_jo'] = df['sent_jo'].apply(lambda x: [i.strip() for i in x])

df['summary'] = df['summary'].apply(lambda x: eval(x))

In [7]:
df['summary'].apply(len) # 요약이 여러개 뽑히는 것도 있음

0       1
1       1
2       1
3       1
4       1
       ..
8995    1
8996    1
8997    2
8998    1
8999    2
Name: summary, Length: 9000, dtype: int64

In [8]:
df['name'].nunique() # 고유값으로 사용 가능함

9000

In [9]:
tmp_df1 = df[['name','article','sent_jo','summary']]

In [10]:
num_article = tmp_df1['article'].apply(len)
num_sent_jo = tmp_df1['sent_jo'].apply(len)

In [11]:
sum(num_article!=num_sent_jo)

0

In [12]:
# make label
import numpy as np
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm

from sentence_transformers import SentenceTransformer

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

model = SentenceTransformer('jhgan/ko-sbert-multitask')

c:\Users\hyj\Anaconda3\envs\virenv39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def make_article(df):
    sent_sum_embed =  df['summary'].apply(lambda x: model.encode(x))
    sent_cn_embed =  df['sent_jo'].apply(lambda x: model.encode(x))

    df['label'] = df['sent_jo']
    for i, (cn_embeds, summ_embeds) in enumerate(zip(sent_cn_embed,sent_sum_embed)):
        label = [0]*(cn_embeds.shape[0])
        for summ_embed in summ_embeds:
            cos_similarity = list(map(lambda x: cos_sim(x,summ_embed), cn_embeds))
            index = np.argmax(cos_similarity)
            if cos_similarity[index] >= 0.80:
                label[index]=1
        df['label'][i] = label
    return df

In [14]:
article_df = make_article(tmp_df1) # summary를 기준으로 조 추출하기 # 17분

C:\Users\hyj\AppData\Local\Temp\ipykernel_4352\410252438.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['sent_jo']
C:\Users\hyj\AppData\Local\Temp\ipykernel_4352\410252438.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = label


In [15]:
num_article = article_df['label'].apply(sum)
drop_row = num_article[num_article==0].index
drop_row

Int64Index([   4,   17,   24,   28,   36,   37,   40,   41,   42,   48,
            ...
            8985, 8986, 8987, 8988, 8990, 8991, 8994, 8995, 8996, 8998],
           dtype='int64', length=2268)

In [16]:
droped_article_df = article_df.drop(drop_row)

In [17]:
num_article = droped_article_df['label'].apply(sum)
drop_row = num_article[num_article==0].index
len(drop_row)

0

In [18]:
droped_article_df.reset_index(drop=True,inplace=True)

In [19]:
num_article.value_counts()

1    6650
2      68
3      10
4       2
6       1
8       1
Name: label, dtype: int64

In [20]:
droped_article_df['extractive'] = droped_article_df['label'].apply(lambda x: np.where(np.array(x) == 1)[0])
droped_article_df['extractive'] = droped_article_df['extractive'].apply(lambda x: x.tolist())

In [21]:
sum(droped_article_df['extractive'].apply(len) == 0 )

0

In [22]:
arti_df = pd.DataFrame(columns = ['name','article','summary'])

for row in range(len(droped_article_df)):
    extractives = droped_article_df['extractive'][row]
    for ext in extractives:
        arti = droped_article_df['article'][row][ext]
        name = droped_article_df['name'][row]
        summary = droped_article_df['summary'][row]
        row_df = {'name': [name], 'article': [arti],'summary':[summary]}
        row_df = pd.DataFrame(data=row_df)
        arti_df = pd.concat([arti_df,row_df])
arti_df.reset_index(inplace=True,drop=True)

In [23]:
arti_df.head() # 추출되여야하는 조와 그에 해당하는 summary

,name,article,summary
0,001_개인정보취급방침,제2조(개인정보의 처리 및 보유기간)\n\n ① 협회는 법령에 따른 개인정보 보유․...,[제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이...
1,001_결혼정보서비스,제3조 (회원가입)\n\n\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사...,[제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...
2,001_보증,제2조(보증금액)\n\n1. 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보...,[제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...
3,001_사이버몰,제25조 (약관 등의 명시와 설명 및 개정)\n\n\n1 몰은 이 약관의 내용과 상...,[제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 ...
4,001_손해보험,제7조(보험금의 지급절차) \n\n\n① 회사는 제6조(보험금의 청구)에서 정한 서...,[제7조(보험금의 지급절차) \n① 회사는 제6조(보험금의 청구)에서 정한 서류를 ...


In [24]:
def make_paragraph(df,col):
    # 항
    list1 = "⓪ ① ② ③ ④ ⑤ ⑥ ⑦ ⑧ ⑨ ⑩ ⑪ ⑫ ⑬ ⑭ ⑮ ⑯ ⑰ ⑱ ⑲ ⑳ ㉑ ㉒ ㉓ ㉔ ㉕ ㉖ ㉗ ㉘ ㉙ ㉚ ㉛ ㉜ ㉝ ㉞ ㉟ ㊱ ㊲ ㊳ ㊴ ㊵ ㊶ ㊷ ㊸ ㊹ ㊺ ㊻ ㊼ ㊽ ㊾ ㊿"
    list1 = list1.split()
    for i in list1:
        df[col] = df[col].apply(lambda x: re.sub(f'{i}',f"?{i}",x))
    # \n제1항
    pattern1 = '\\n[제|1-9|항]{1,}'
    matched_list = df[col].apply(lambda x: re.findall(pattern1,x))
    matched_list = matched_list.apply(lambda x: set(x))
    matched_list = matched_list.apply(lambda x: list(x))

    for i in tqdm(range(len(df))):
        text = df[col][i]
        matchedList = matched_list[i]
        for match in matchedList:
            text = text.replace(match,'\n?'+match)
        df[col][i] = text
        
    return df

In [25]:
def make_subparagraph(df,col):
    # 호 
    # \n1 -> \n?\n1.
    # \n 1 -> \n?\n 1.
    pattern1 = '\\n[ ]{0,}[1-9]{1,}'
    # pattern1 = '\\n[ ]{0,}[1-9]{1,}\.'
    matched_list = df[col].apply(lambda x: re.findall(pattern1,x))
    matched_list = matched_list.apply(lambda x: set(x))
    matched_list = matched_list.apply(lambda x: list(x))

    for i in tqdm(range(len(df))):
        text = df[col][i]
        matchedList = matched_list[i]
        for match in matchedList:
            text = text.replace(match,'\n?'+match)
        df[col][i] = text
    
    # \n제1호
    pattern1 = '\\n[제|1-9|호]{1,}'
    matched_list = df[col].apply(lambda x: re.findall(pattern1,x))
    matched_list = matched_list.apply(lambda x: set(x))
    matched_list = matched_list.apply(lambda x: list(x))

    for i in tqdm(range(len(df))):
        text = df[col][i]
        matchedList = matched_list[i]
        for match in matchedList:
            text = text.replace(match,'\n?'+match)
        df[col][i] = text
    return df

In [26]:
# 목
def make_item(df,col):
    list2 = "가 나 다 라 마 바 사 아 자 차 카 타 파 하 거 너 더 러 머 버 서 어 저 처 커 터 퍼 허"
    list3 = "A B C D E F G H I J K L M N O P Q R S T U V W X Y Z"
    list2 = list2.split()
    list3 = list3.split()

    for i in tqdm(list2):
        # 가. 나. 다.
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}]\.',f"\n?{i}.",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}]\.',f"\n?{i}.",x))    
    for i in tqdm(list3):
        # A. B.
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}]\.',f"\n?{i}.",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}]\.',f"\n?{i}.",x))
    return df

In [27]:
def make_rest(df,col):
    list2 = "가 나 다 라 마 바 사 아 자 차 카 타 파 하 거 너 더 러 머 버 서 어 저 처 커 터 퍼 허"
    list2 = list2.split()
    
    for i in tqdm(list2):
    # 세분화
        # 가), 나), 다)
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}]\)',f"\n?{i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}]\)',f"\n?{i})",x))
        # 가 ), 나 ), 다 )
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}] \)',f"\n?{i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}] \)',f"\n?{i})",x))
        # (가), (나), (다)
        df[col] = df[col].apply(lambda x: re.sub(f'\\n\([{i}]\)',f"\n?({i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n \([{i}]\)',f"\n?({i})",x))
        # ( 가 ), ( 나 ), ( 다 )
        df[col] = df[col].apply(lambda x: re.sub(f'\\n\( [{i}] \)',f"\n?({i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n \( [{i}] \)',f"\n?({i})",x))
    # 세분화
    for i in range(50):
        # 1), 2), 3),
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}]\)',f"\n?{i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}]\)',f"\n?{i})",x))
        # 1 ), 2 ), 3 ),
        df[col] = df[col].apply(lambda x: re.sub(f'\\n[{i}] \)',f"\n?{i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n [{i}] \)',f"\n?{i})",x))
        # (1), (2), (3),
        df[col] = df[col].apply(lambda x: re.sub(f'\\n\([{i}]\)',f"\n?({i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n \([{i}]\)',f"\n?({i})",x))
        # ( 1 ), ( 2 ), ( 3 ),
        df[col] = df[col].apply(lambda x: re.sub(f'\\n\( [{i}] \)',f"\n?({i})",x))
        df[col] = df[col].apply(lambda x: re.sub(f'\\n \( [{i}] \)',f"\n?({i})",x))
    return df

In [28]:
arti_df = make_paragraph(arti_df,'article')

100%|██████████| 6838/6838 [00:01<00:00, 5182.25it/s]


In [29]:
arti_df = make_subparagraph(arti_df,'article')

100%|██████████| 6838/6838 [00:01<00:00, 5225.83it/s]


In [30]:
arti_df = make_item(arti_df,'article')

100%|██████████| 26/26 [00:02<00:00,  9.15it/s]


In [31]:
arti_df = make_rest(arti_df,'article')

100%|██████████| 28/28 [00:12<00:00,  2.26it/s]


In [32]:
arti_df['article_sent'] = arti_df['article'].apply(lambda x: x.replace('\n',''))
arti_df['article_sent'] = arti_df['article_sent'].apply(lambda x: x.split('?'))

arti_df['summary'] = arti_df['summary'].apply(lambda x: '\n'.join(x))
arti_df = make_paragraph(arti_df,'summary')
arti_df = make_subparagraph(arti_df,'summary')
arti_df = make_item(arti_df,'summary')
arti_df = make_rest(arti_df,'summary')
arti_df['summary_sent'] = arti_df['summary'].apply(lambda x: x.replace('\n',''))
arti_df['summary_sent'] = arti_df['summary_sent'].apply(lambda x: x.split('?'))

100%|██████████| 28/28 [00:12<00:00,  2.21it/s]


In [33]:
arti_df['article_sent'] = arti_df['article_sent'].apply(lambda x: [i.strip() for i in x])
arti_df['article_sent'] = arti_df['article_sent'].apply(lambda x: [i for i in x if i not in {''}])

arti_df['summary_sent'] = arti_df['summary_sent'].apply(lambda x: [i.strip() for i in x])
arti_df['summary_sent'] = arti_df['summary_sent'].apply(lambda x: [i for i in x if i not in {''}])

In [34]:
# 4분 45초
article_embed = arti_df['article_sent'].apply(lambda x: model.encode(x))
summary_embed = arti_df['summary_sent'].apply(lambda x: model.encode(x))

arti_df['label'] = arti_df['article_sent']
for i, (arti_embeds, summ_embeds) in enumerate(zip(article_embed,summary_embed)):
    label = [0]*(arti_embeds.shape[0])
    for summ_embed in summ_embeds:
        cos_similarity = list(map(lambda x: cos_sim(x,summ_embed), arti_embeds))
        index = np.argmax(cos_similarity)
        if cos_similarity[index] >= 0.85:
            label[index]=1
    arti_df['label'][i] = label

In [35]:
arti_df

,name,article,summary,article_sent,summary_sent,label
0,001_개인정보취급방침,제2조(개인정보의 처리 및 보유기간)\n\n ?① 협회는 법령에 따른 개인정보 보유...,제2조(개인정보의 처리 및 보유기간) \n?① 협회는 법령에 따른 개인정보 보유․이...,"[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[1, 1, 0, 0]"
1,001_결혼정보서비스,제3조 (회원가입)\n\n\n?① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회...,제3조 (회원가입)\n?① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...,"[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[1, 1]"
2,001_보증,제2조(보증금액)\n\n?\n?\n?\n1. 이 보증서에 의한 보증금액은 채권자의 ...,제2조(보증금액)\n ?① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...,"[제2조(보증금액), 1. 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...","[제2조(보증금액), ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...","[1, 1]"
3,001_사이버몰,제25조 (약관 등의 명시와 설명 및 개정)\n\n\n?\n?\n?\n1 몰은 이 ...,제3조 (약관 등의 명시와 설명 및 개정)\n?① 몰은 이 약관의 내용과 상호 및 ...,"[제25조 (약관 등의 명시와 설명 및 개정), 1 몰은 이 약관의 내용과 상호 및...","[제3조 (약관 등의 명시와 설명 및 개정), ① 몰은 이 약관의 내용과 상호 및 ...","[0, 1]"
4,001_손해보험,제7조(보험금의 지급절차) \n\n\n?① 회사는 제6조(보험금의 청구)에서 정한 ...,제7조(보험금의 지급절차) \n?① 회사는 제6조(보험금의 청구)에서 정한 서류를 ...,"[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[1, 1, 0]"
...,...,...,...,...,...,...
6833,514_임대차계약,제6조 (계약해제 및 위약금)\n\n?\n?\n?\n1. 을이 아래 각 호 1에 해...,"제30조 (명도대행)\n?\n?\n제1항 갑이 본 계약 종료 후에도 자기 소유물, ...","[제6조 (계약해제 및 위약금), 1. 을이 아래 각 호 1에 해당할 경우에는 을은...","[제30조 (명도대행), 제1항 갑이 본 계약 종료 후에도 자기 소유물, 시설물 등...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6834,517_임대차계약,제6조 (계약해제 및 위약금)\n\n?\n?\n?\n1. 을이 아래 각 호 1에 해...,제10조(계약의 해지)\n?\n?\n?\n제2항 다음 각 항에 해당하는 사유가 발생...,"[제6조 (계약해제 및 위약금), 1. 을이 아래 각 호 1에 해당할 경우에는 을은...","[제10조(계약의 해지), 제2항 다음 각 항에 해당하는 사유가 발생한 경우 을은 ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6835,625_임대차계약,제6조 (계약해제 및 위약금)\n\n?\n?\n?\n1. 을이 아래 각 호 1에 해...,제6조(계약해제권 행사 완화)\n?\n?\n제1항 본계약 및 관리규정상의 업무를 불...,"[제6조 (계약해제 및 위약금), 1. 을이 아래 각 호 1에 해당할 경우에는 을은...","[제6조(계약해제권 행사 완화), 제1항 본계약 및 관리규정상의 업무를 불이행 또는...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6836,629_임대차계약,제2조 (임 대 료)\n\n?\n?\n?\n1. 을은 월 임대료를 매월 일까지 갑에...,제2조 (임대료의 기산등)\n?\n?\n제1항 이계약에 의한 임대료는 다른 약정이 ...,"[제2조 (임 대 료), 1. 을은 월 임대료를 매월 일까지 갑에게 당월 분 임대료...","[제2조 (임대료의 기산등), 제1항 이계약에 의한 임대료는 다른 약정이 없는 한 ...","[0, 0, 1, 0, 0, 0, 0]"


In [36]:
drop_index = arti_df.loc[arti_df['label'].apply(sum)==0].index
print(len(drop_index))
arti_df.drop(drop_index,inplace=True)

236


In [37]:
arti_df.loc[arti_df['label'].apply(sum)==0]

,name,article,summary,article_sent,summary_sent,label


In [38]:
arti_df.reset_index(drop=True,inplace=True)

In [39]:
arti_df

,name,article,summary,article_sent,summary_sent,label
0,001_개인정보취급방침,제2조(개인정보의 처리 및 보유기간)\n\n ?① 협회는 법령에 따른 개인정보 보유...,제2조(개인정보의 처리 및 보유기간) \n?① 협회는 법령에 따른 개인정보 보유․이...,"[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[1, 1, 0, 0]"
1,001_결혼정보서비스,제3조 (회원가입)\n\n\n?① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회...,제3조 (회원가입)\n?① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...,"[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[1, 1]"
2,001_보증,제2조(보증금액)\n\n?\n?\n?\n1. 이 보증서에 의한 보증금액은 채권자의 ...,제2조(보증금액)\n ?① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...,"[제2조(보증금액), 1. 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...","[제2조(보증금액), ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...","[1, 1]"
3,001_사이버몰,제25조 (약관 등의 명시와 설명 및 개정)\n\n\n?\n?\n?\n1 몰은 이 ...,제3조 (약관 등의 명시와 설명 및 개정)\n?① 몰은 이 약관의 내용과 상호 및 ...,"[제25조 (약관 등의 명시와 설명 및 개정), 1 몰은 이 약관의 내용과 상호 및...","[제3조 (약관 등의 명시와 설명 및 개정), ① 몰은 이 약관의 내용과 상호 및 ...","[0, 1]"
4,001_손해보험,제7조(보험금의 지급절차) \n\n\n?① 회사는 제6조(보험금의 청구)에서 정한 ...,제7조(보험금의 지급절차) \n?① 회사는 제6조(보험금의 청구)에서 정한 서류를 ...,"[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[1, 1, 0]"
...,...,...,...,...,...,...
6597,353_분양계약,제7조(시설물의 유지 관리)\n\n?\n?\n제2항. 갑은 단지 관리상 필요하다고 ...,제7조(시설물의 유지 관리)\n갑은 단지 관리상 필요하다고 인정할 때에는 을에게 사...,"[제7조(시설물의 유지 관리), 제2항. 갑은 단지 관리상 필요하다고 인정할 때에는...",[제7조(시설물의 유지 관리)갑은 단지 관리상 필요하다고 인정할 때에는 을에게 사전...,"[0, 1]"
6598,354_분양계약,제8조(해제)\n\n?\n?\n제4항. 갑은 이 계약이 해제되었을 때 을이 기불입...,"제8조(해제)\n갑은 이 계약이 해제되었을 때 을이 기불입한 전납금, 융자금에 대한...","[제8조(해제), 제4항. 갑은 이 계약이 해제되었을 때 을이 기불입한 전납금, 융...","[제8조(해제)갑은 이 계약이 해제되었을 때 을이 기불입한 전납금, 융자금에 대한 ...","[0, 1]"
6599,448_임대차계약,"제18조 (갑의 면책사항)\n\n?\n?\n?\n1. 도난, 천재지변 기타 불가항력...","제18조\n?\n?\n제2항 도난,화재 기타 불가항력적인 사고로 발생되는 일체...","[제18조 (갑의 면책사항), 1. 도난, 천재지변 기타 불가항력적 사유로 인한 사...","[제18조, 제2항 도난,화재 기타 불가항력적인 사고로 발생되는 일체의 손실에...","[0, 1, 0]"
6600,458_임대차계약,제20조(갑의 면책사항)\n\n?\n?\n제1항 을은 목적 건물내에서 발생한 모든 ...,제8조(갑의 면책사항)\n?\n?\n제1항 을은 목적 건물내에서 발생한 모든 사고에...,"[제20조(갑의 면책사항), 제1항 을은 목적 건물내에서 발생한 모든 사고에 대해서...","[제8조(갑의 면책사항), 제1항 을은 목적 건물내에서 발생한 모든 사고에 대해서 ...","[0, 1]"


In [43]:
arti_df.to_csv('./preprocessed_data/sentence_data.csv')

### 데이터 형식에 맞게 저장

In [4]:
# kobertsum
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('./preprocessed_data/data.csv',index_col=[0])
arti_df = pd.read_csv('./preprocessed_data/sentence_data.csv',index_col=[0])
arti_df['article_sent'] = arti_df['article_sent'].apply(lambda x: eval(x))
arti_df['summary_sent'] = arti_df['summary_sent'].apply(lambda x: eval(x))
arti_df['label'] = arti_df['label'].apply(lambda x: eval(x))

In [5]:
kobertsum_df = pd.merge(arti_df[['name','article_sent','summary_sent','label']],df[['category','name','ad_label']],on='name',how='left')

In [6]:
kobertsum_df['extractive'] = kobertsum_df['label'].apply(lambda x: np.where(np.array(x) == 1)[0])
kobertsum_df['extractive'] = kobertsum_df['extractive'].apply(lambda x: x.tolist())

In [7]:
kobertsum_df = kobertsum_df[['category','name','ad_label','article_sent','summary_sent','extractive']]
kobertsum_df.columns = ['media','id','ad_label','article_original','extractive_sents','extractive']
kobertsum_df

,media,id,ad_label,article_original,extractive_sents,extractive
0,24. 개인정보취급방침,001_개인정보취급방침,1,"[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[제2조(개인정보의 처리 및 보유기간), ① 협회는 법령에 따른 개인정보 보유․이용...","[0, 1]"
1,27. 결혼정보서비스,001_결혼정보서비스,1,"[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[제3조 (회원가입), ① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제...","[0, 1]"
2,30.보증,001_보증,1,"[제2조(보증금액), 1. 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증...","[제2조(보증금액), ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...","[0, 1]"
3,31. 사이버몰,001_사이버몰,1,"[제25조 (약관 등의 명시와 설명 및 개정), 1 몰은 이 약관의 내용과 상호 및...","[제3조 (약관 등의 명시와 설명 및 개정), ① 몰은 이 약관의 내용과 상호 및 ...",[1]
4,13. 손해보험,001_손해보험,1,"[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[제7조(보험금의 지급절차), ① 회사는 제6조(보험금의 청구)에서 정한 서류를 접...","[0, 1]"
...,...,...,...,...,...,...
6597,3. 분양계약_PDF,353_분양계약,2,"[제7조(시설물의 유지 관리), 제2항. 갑은 단지 관리상 필요하다고 인정할 때에는...",[제7조(시설물의 유지 관리)갑은 단지 관리상 필요하다고 인정할 때에는 을에게 사전...,[1]
6598,3. 분양계약_PDF,354_분양계약,2,"[제8조(해제), 제4항. 갑은 이 계약이 해제되었을 때 을이 기불입한 전납금, 융...","[제8조(해제)갑은 이 계약이 해제되었을 때 을이 기불입한 전납금, 융자금에 대한 ...",[1]
6599,5. 임대차계약,448_임대차계약,2,"[제18조 (갑의 면책사항), 1. 도난, 천재지변 기타 불가항력적 사유로 인한 사...","[제18조, 제2항 도난,화재 기타 불가항력적인 사고로 발생되는 일체의 손실에...",[1]
6600,5. 임대차계약,458_임대차계약,2,"[제20조(갑의 면책사항), 제1항 을은 목적 건물내에서 발생한 모든 사고에 대해서...","[제8조(갑의 면책사항), 제1항 을은 목적 건물내에서 발생한 모든 사고에 대해서 ...",[1]


In [8]:
kob_train,kob_test = train_test_split(kobertsum_df,test_size=0.2,shuffle=True,random_state=45)
kob_valid,kob_test = train_test_split(kob_test,test_size=0.5,shuffle=True,random_state=45)

In [9]:
print(len(kob_train))
print(len(kob_valid))
print(len(kob_test))

5281
660
661


In [10]:
kob_train

,media,id,ad_label,article_original,extractive_sents,extractive
1516,24. 개인정보취급방침,083_개인정보취급방침,1,"[제1조 수집한 개인정보의 이용000 및 000 관련 제반 서비스(모바일 웹, 앱 ...",[제1조(개인정보의 처리목적)(개인정보처리자명)은(는) 다음의 목적을 위하여 개인정...,[1]
2539,11. 전자금융거래,140_전자금융거래,1,"[제17조(분쟁처리 및 분쟁조정), ③ 제1항에 따라 고객이 회사에 손해배상 등 분...","[제18조(분쟁처리 및 분쟁조정), ③ 제1항에 따라 고객이 회사에 손해배상 등 분...",[1]
3034,16.자동차보험,184_자동차보험,1,"[제49조 (피보험자동차의 교체), ① 보험계약자 또는 기명피보험자가 보험기간 중에...","[제49조 (피보험자동차의 교체), ⑤ 보험회사가 제1항의 승인을 거절한 경우 교체...","[0, 6]"
3453,11. 전자금융거래,237_전자금융거래,1,"[제30조(약관의 변경 등), ② 제1항에 불구하고 회사가 법령의 개정으로 인하여 ...","[제16조(약관의 변경 등), ② 제1항에 불구하고 회사가 법령의 개정으로 인하여 ...",[1]
2088,13. 손해보험,112_손해보험,1,[1 제27조(보험료의 납입이 연체되는 경우 납입최고(독촉)와 계약의 해지)에 따라...,"[제28조(보험료의 납입연체로 인한 해지계약의 부활(효력회복)), ① 제27조(보험...",[0]
...,...,...,...,...,...,...
4703,2. 공급계약,052_공급계약,2,"[제 9 조 (계약의 해지), 1. 갑은 을이 본 계약서에 정하는 사항을 위반하거나...","[제2조(계약의 해제 및 해지), 제1항 갑은 을이 본 계약서에 정하는 사항을 위반...",[1]
3616,39. 체육시설,262_체육시설,1,"[제 8 조(요금의 환불), 1. 입장절차를 마친 이용자가 경기 전 임의로 이용계약...","[제8조(요금의 환불), ① 입장절차를 마친 이용자가 경기 전 임의로 이용계약을 취...","[0, 1]"
6012,31. 사이버몰,080_사이버몰,1,"[제 26조(회원가입), ② 몰 은 제1항과 같이 회원으로 가입할 것을 신청한 이용...","[제6조(회원가입), ② 몰 은 제1항과 같이 회원으로 가입할 것을 신청한 이용자...","[1, 2]"
5763,2. 공급계약,361_공급계약,2,"[제19조(면적 및 조건변경), 1항. 본 계약은 갑이 제시한 도면에 의한 구좌당 ...","[제4조(면적 및 조건변경), 1항. 본 계약은 갑이 제시한 도면에 의한 구좌당 계...",[1]


In [11]:
mem_train = kob_train[['media','id','article_original','extractive_sents']]
mem_train.columns = ['category','name','sent_cn','sent_summ']

mem_valid = kob_valid[['media','id','article_original','extractive_sents']]
mem_valid.columns = ['category','name','sent_cn','sent_summ']

mem_test = kob_test[['media','id','article_original','extractive_sents']]
mem_test.columns = ['category','name','sent_cn','sent_summ']

In [12]:
mem_train.head(1)

,category,name,sent_cn,sent_summ
1516,24. 개인정보취급방침,083_개인정보취급방침,"[제1조 수집한 개인정보의 이용000 및 000 관련 제반 서비스(모바일 웹, 앱 ...",[제1조(개인정보의 처리목적)(개인정보처리자명)은(는) 다음의 목적을 위하여 개인정...


In [13]:
mem_train.to_csv('./preprocessed_data//mem_train.csv')
mem_valid.to_csv('./preprocessed_data//mem_valid.csv')
mem_test.to_csv('./preprocessed_data//mem_test.csv')

In [14]:
from sklearn.preprocessing import LabelEncoder
df = kobertsum_df
le = LabelEncoder()
le = le.fit(df['id'])

kob_train['id'] = le.transform(kob_train['id'])
kob_valid['id'] = le.transform(kob_valid['id'])
kob_test['id'] = le.transform(kob_test['id'])

kob_train.drop(['ad_label','extractive_sents'],axis=1,inplace=True)
kob_valid.drop(['ad_label','extractive_sents'],axis=1,inplace=True)
kob_test.drop(['ad_label','extractive_sents'],axis=1,inplace=True)

In [15]:
kob_train.head(1)

,media,id,article_original,extractive
1516,24. 개인정보취급방침,2332,"[제1조 수집한 개인정보의 이용000 및 000 관련 제반 서비스(모바일 웹, 앱 ...",[1]


In [27]:
jsonl_path = "./preprocessed_data/kobertsum_train.jsonl"

df = kob_train
file_to_write = ""
for index in df.index:
    df.loc[index].to_json("./tmp_data/row{}.json".format(index),force_ascii=False)
    with open("./tmp_data/row{}.json".format(index)) as file_handle:
        file_content = file_handle.read()
        file_to_write += file_content + "\n"
        
with open(jsonl_path,"w") as file_handle:
    file_handle.write(file_to_write)

jsonl_path = "./preprocessed_data/kobertsum_valid.jsonl"

df = kob_valid
file_to_write = ""
for index in df.index:
    df.loc[index].to_json("./tmp_data/row{}.json".format(index),force_ascii=False)
    with open("./tmp_data/row{}.json".format(index)) as file_handle:
        file_content = file_handle.read()
        file_to_write += file_content + "\n"
        
with open(jsonl_path,"w") as file_handle:
    file_handle.write(file_to_write)
    
jsonl_path = "./preprocessed_data/kobertsum_test.jsonl"

df = kob_test
file_to_write = ""
for index in df.index:
    df.loc[index].to_json("./tmp_data/row{}.json".format(index),force_ascii=False)
    with open("./tmp_data/row{}.json".format(index)) as file_handle:
        file_content = file_handle.read()
        file_to_write += file_content + "\n"
        
with open(jsonl_path,"w") as file_handle:
    file_handle.write(file_to_write)
    